## Import packages

In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

## Step I. Load data

In [48]:
train = pd.read_csv("train_original.csv")
test = pd.read_csv("test_original.csv")

## Step II. Explore and prepare data

In [49]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [50]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [52]:
X_train = train.iloc[:, 1:-1]
y_train = train.iloc[:, -1]
X_test = test.iloc[:, 1:]

In [53]:
X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [54]:
y_train.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [55]:
X_train.describe(include="all")

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
count,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,1460,...,1460.000000,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460
unique,NaN,5,NaN,NaN,2,2,4,4,2,5,...,NaN,NaN,3,4,4,NaN,NaN,NaN,9,6
top,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,NaN,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal
freq,NaN,1151,NaN,NaN,1454,50,925,1311,1459,1052,...,NaN,NaN,3,157,49,NaN,NaN,NaN,1267,1198
mean,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,NaN,...,15.060959,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN
std,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,NaN,...,55.757415,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN
min,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN
25%,20.000000,NaN,NaN,7553.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN
50%,50.000000,NaN,NaN,9478.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN
75%,70.000000,NaN,NaN,11601.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN


In [56]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

### Extract categorical columns

In [57]:
col_grps=X_train.columns.to_series().groupby(train.dtypes).groups

In [58]:
col_grps

{dtype('int64'): ['MSSubClass',
  'LotArea',
  'OverallQual',
  'OverallCond',
  'YearBuilt',
  'YearRemodAdd',
  'BsmtFinSF1',
  'BsmtFinSF2',
  'BsmtUnfSF',
  'TotalBsmtSF',
  '1stFlrSF',
  '2ndFlrSF',
  'LowQualFinSF',
  'GrLivArea',
  'BsmtFullBath',
  'BsmtHalfBath',
  'FullBath',
  'HalfBath',
  'BedroomAbvGr',
  'KitchenAbvGr',
  'TotRmsAbvGrd',
  'Fireplaces',
  'GarageCars',
  'GarageArea',
  'WoodDeckSF',
  'OpenPorchSF',
  'EnclosedPorch',
  '3SsnPorch',
  'ScreenPorch',
  'PoolArea',
  'MiscVal',
  'MoSold',
  'YrSold'],
 dtype('float64'): ['LotFrontage', 'MasVnrArea', 'GarageYrBlt'],
 dtype('O'): ['MSZoning',
  'Street',
  'Alley',
  'LotShape',
  'LandContour',
  'Utilities',
  'LotConfig',
  'LandSlope',
  'Neighborhood',
  'Condition1',
  'Condition2',
  'BldgType',
  'HouseStyle',
  'RoofStyle',
  'RoofMatl',
  'Exterior1st',
  'Exterior2nd',
  'MasVnrType',
  'ExterQual',
  'ExterCond',
  'Foundation',
  'BsmtQual',
  'BsmtCond',
  'BsmtExposure',
  'BsmtFinType1',
  

In [59]:
col_grps.keys()

[dtype('O'), dtype('int64'), dtype('float64')]

In [60]:
cat_feats = col_grps[col_grps.keys()[0]]

In [61]:
cat_feats.append('MSSubClass')

In [62]:
cat_feats

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition',
 'MSSubClass']

### Function for imputation

In [63]:
def impute(dataFrame):
    df = dataFrame.copy()
    df.MSZoning.fillna("NA", inplace=True)    
    df.LotFrontage.fillna(0, inplace=True)    
    df.Alley.fillna("NA", inplace=True)    
    df.Utilities.fillna("NA", inplace=True)    
    df.Exterior1st.fillna("Other", inplace=True)    
    df.Exterior2nd.fillna("Other", inplace=True)    
    df.MasVnrType.fillna("None", inplace=True)
    df.MasVnrArea.fillna(0, inplace=True)
    df.BsmtQual.fillna("NA", inplace=True)
    df.BsmtCond.fillna("NA", inplace=True)
    df.BsmtExposure.fillna("NA", inplace=True)
    df.BsmtFinType1.fillna("NA", inplace=True)    
    df.BsmtFinSF1.fillna(0, inplace=True)    
    df.BsmtFinType2.fillna("NA", inplace=True)    
    df.BsmtFinSF2.fillna(0, inplace=True)    
    df.BsmtUnfSF.fillna(0, inplace=True)    
    df.TotalBsmtSF.fillna(0, inplace=True)    
    df.Electrical.fillna("FuseP", inplace=True)    
    df.BsmtFullBath.fillna(0, inplace=True)    
    df.BsmtHalfBath.fillna(0, inplace=True)    
    df.KitchenQual.fillna("Po", inplace=True)    
    df.Functional.fillna("Sal", inplace=True)    
    df.FireplaceQu.fillna("NA", inplace=True)
    df.GarageType.fillna("NA", inplace=True)
    df.GarageYrBlt.fillna(1000, inplace=True)
    df.GarageFinish.fillna("NA", inplace=True)    
    df.GarageCars.fillna(0, inplace=True)    
    df.GarageArea.fillna(0, inplace=True)    
    df.GarageQual.fillna("NA", inplace=True)
    df.GarageCond.fillna("NA", inplace=True)
    df.PoolQC.fillna("NA", inplace=True)
    df.Fence.fillna("NA", inplace=True)
    df.MiscFeature.fillna("NA", inplace=True)    
    df.SaleType.fillna("Oth", inplace=True) 
    return df

### Function for obtaining dummies from categorical features

In [64]:
def categor(train, test):
    df = pd.concat([train, test], axis=0)
    df = pd.get_dummies(df, columns=cat_feats, drop_first=True)
    train = df.iloc[:train.shape[0], :]
    test = df.iloc[train.shape[0]:, :]
    return (train, test)

### Function for feature extraction and feature removal

In [65]:
def feat_ext(dataFrame):
    df = dataFrame.copy()
    df['Age'] = 2016-df['YearRemodAdd']
    df['GarageAge'] = 2016-df['GarageYrBlt']
    df['SoldSinceYr'] = 2016-df['YrSold']
    df = df.drop(['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold'], axis=1)
    return df

### Function for standardization

In [66]:
def standardize(train, test):
    scaler = StandardScaler().fit(train)
    train = pd.DataFrame(scaler.transform(train), columns=train.columns)
    test = pd.DataFrame(scaler.transform(test), columns=train.columns)
    return (train, test)

In [67]:
X_train_imputed = impute(X_train)
X_test_imputed = impute(X_test)
X_train_categorical, X_test_categorical = categor(X_train_imputed, X_test_imputed)
X_train_modified = feat_ext(X_train_categorical)
X_test_modified = feat_ext(X_test_categorical)

In [68]:
X_train_mod, X_test_mod = standardize(X_train_modified, X_test_modified)

In [69]:
print "No. of NaN's in train: ", X_train.isnull().sum().sum()
print "No. of NaN's in train_imputed: ", X_train_imputed.isnull().sum().sum()
print "No. of NaN's in test: ", X_test.isnull().sum().sum()
print "No. of NaN's in test_imputed: ", X_test_imputed.isnull().sum().sum()

No. of NaN's in train:  6965
No. of NaN's in train_imputed:  0
No. of NaN's in test:  7000
No. of NaN's in test_imputed:  0


In [70]:
X_train_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 no

In [71]:
X_test_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
MSSubClass       1459 non-null int64
MSZoning         1459 non-null object
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            1459 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1459 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1459 non-null object
Exterior2nd      1459 no

In [72]:
X_train_categorical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Columns: 278 entries, LotFrontage to MSSubClass_190
dtypes: float64(254), int64(24)
memory usage: 3.1 MB


In [73]:
X_test_categorical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Columns: 278 entries, LotFrontage to MSSubClass_190
dtypes: float64(254), int64(24)
memory usage: 3.1 MB


In [74]:
X_train_modified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Columns: 276 entries, LotFrontage to SoldSinceYr
dtypes: float64(254), int64(22)
memory usage: 3.1 MB


In [75]:
X_test_modified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Columns: 276 entries, LotFrontage to SoldSinceYr
dtypes: float64(254), int64(22)
memory usage: 3.1 MB


In [76]:
X_train_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 276 entries, LotFrontage to SoldSinceYr
dtypes: float64(276)
memory usage: 3.1 MB


In [77]:
X_test_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 276 entries, LotFrontage to SoldSinceYr
dtypes: float64(276)
memory usage: 3.1 MB


### Merge 'X_train' and 'y_train' into 'train'.

In [80]:
train = pd.concat([X_train_mod, y_train], axis=1)

In [82]:
train.head()

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,Age,GarageAge,SoldSinceYr,SalePrice
0,0.212877,-0.207142,0.651479,-0.517200,0.514104,0.575425,-0.288653,-0.944591,-0.459303,-0.793434,...,-0.192177,-0.251724,0.0,-0.21236,-0.083045,-0.144841,-0.878668,-0.349711,-0.138777,208500
1,0.645747,-0.091886,-0.071836,2.179628,-0.570750,1.171992,-0.288653,-0.641228,0.466465,0.257140,...,-0.192177,-0.251724,0.0,-0.21236,-0.083045,-0.144841,0.429577,-0.229857,0.614439,181500
2,0.299451,0.073480,0.651479,-0.517200,0.325915,0.092907,-0.288653,-0.301643,-0.313369,-0.627826,...,-0.192177,-0.251724,0.0,-0.21236,-0.083045,-0.144841,-0.830215,-0.340833,-0.138777,223500
3,0.068587,-0.096897,0.651479,-0.517200,-0.570750,-0.499274,-0.288653,-0.061670,-0.687324,-0.521734,...,-0.192177,-0.251724,0.0,-0.21236,-0.083045,-0.144841,0.720298,-0.327516,1.367655,140000
4,0.761179,0.375148,1.374795,-0.517200,1.366489,0.463568,-0.288653,-0.174865,0.199680,-0.045611,...,-0.192177,-0.251724,0.0,-0.21236,-0.083045,-0.144841,-0.733308,-0.336394,-0.138777,250000


### Save modified dataset

In [83]:
train.to_csv('train.csv', index=0)
X_test_mod.to_csv('X_test.csv', index=0)